Started with pandas.
Imported invoice dataset.
Check first few rows.

In [40]:
import pandas as pd
import numpy as np

# Load invoice data
invoice_df = pd.read_csv("Data/invoice.csv")

# Preview
print(invoice_df.head())

   id        date  tarif_type  counter_statue  reading_remarque  \
0   0   24/3/2014          11               0                 8   
1   0   29/3/2013          11               0                 6   
2   0   23/3/2015          11               0                 8   
3   0   13/7/2015          11               0                 8   
4   0  17/11/2016          11               0                 9   

   consommation_level_4  months_number counter_type  counter_coefficient  \
0                     0              4         ELEC                    1   
1                     0              4         ELEC                    1   
2                     0              4         ELEC                    1   
3                     0              4         ELEC                    1   
4                     0             12         ELEC                    1   

   consommation_level_1  consommation_level_2  consommation_level_3  
0                    82                     0                     0  


Check for empty data:

In [42]:
print(invoice_df.isnull().sum())

id                      0
date                    0
tarif_type              0
counter_statue          0
reading_remarque        0
consommation_level_4    0
months_number           0
counter_type            0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
dtype: int64


No empty data detected. 
Now, we parse dates as datetime objects. 
This ensures date data is processed correctly afterwards.

In [43]:
# Convert date columns to datetime
invoice_df['date'] = pd.to_datetime(invoice_df['date'])

print(invoice_df['date'].head())

0   2014-03-24
1   2013-03-29
2   2015-03-23
3   2015-07-13
4   2016-11-17
Name: date, dtype: datetime64[ns]


/var/folders/hx/f9l8cr254_905tmd749bc8sw0000gn/T/ipykernel_5066/1043635998.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  invoice_df['date'] = pd.to_datetime(invoice_df['date'])


We intend to classify clients, so we tried to summarise invoice data by client. For each client, we computed rough statistics such as the mean , date and sum for our focus on consumption levels.

In [47]:
# Aggregating invoice data by client 'id'
consumption_stats = invoice_df.groupby('id').agg({
    'consommation_level_1': ['sum', 'mean', 'max', 'std'],
    'consommation_level_2': ['sum', 'mean', 'max', 'std'],
    'consommation_level_3': ['sum', 'mean', 'max', 'std'],
    'consommation_level_4': ['sum', 'mean', 'max', 'std'],
    'date': ['min', 'max'],     # Add earliest and latest invoice date
    'counter_statue': 'count',  # How many invoices per client
}).reset_index()

consumption_stats.columns = ['_'.join(col).strip() if col[1] else col[0] for col in consumption_stats.columns.values]

consumption_stats['elapsed_time'] = (consumption_stats['date_max'] - consumption_stats['date_min']).dt.days

# Preview the data
print(consumption_stats.head())

    id  consommation_level_1_sum  consommation_level_1_mean  \
0    0                     12334                 352.400000   
1    1                     20629                 557.540541   
2   10                     14375                 798.611111   
3   12                      5724                 357.750000   
4  100                        24                   1.200000   

   consommation_level_1_max  consommation_level_1_std  \
0                      1200                310.343472   
1                      1207                197.935960   
2                      2400                513.841374   
3                       925                232.908995   
4                        15                  3.607011   

   consommation_level_2_sum  consommation_level_2_mean  \
0                       370                  10.571429   
1                         0                   0.000000   
2                       682                  37.888889   
3                      1740                 10

Now we highlight possible anomalous clients. Could be fraud?

In [49]:
# Calculate IQR for consumption_level_1
Q1 = consumption_stats['consommation_level_1_sum'].quantile(0.25)
Q3 = consumption_stats['consommation_level_1_sum'].quantile(0.75)
IQR = Q3 - Q1

# Define anomaly boundaries using IQR
lower_bound = Q1 - 1.5 * IQR  # Lower bound for anomalies
upper_bound = Q3 + 1.5 * IQR  # Upper bound for anomalies

# Flagging anomalies based on IQR for consumption level 1
consumption_stats['anomaly_flag'] = np.where(
    (consumption_stats['consommation_level_1_sum'] < lower_bound) |  # Below the lower bound (too low)
    (consumption_stats['consommation_level_1_sum'] > upper_bound),   # Above the upper bound (too high)
    1,  # Mark as anomaly
    0   # Mark as normal
)

# Merge the anomaly flags back to the original invoice data based on 'id'
invoice_with_anomalies = pd.merge(invoice_df, consumption_stats[['id', 'anomaly_flag']], on='id', how='left')

# Save the updated dataset (with anomalies) back to the original invoice.csv file
invoice_with_anomalies.to_csv("Data/invoice.csv", index=False)

# Preview the updated dataset
print("Updated invoice data with anomaly flag based on IQR:")
print(invoice_with_anomalies.head())

Updated invoice data with anomaly flag based on IQR:
   id       date  tarif_type  counter_statue  reading_remarque  \
0   0 2014-03-24          11               0                 8   
1   0 2013-03-29          11               0                 6   
2   0 2015-03-23          11               0                 8   
3   0 2015-07-13          11               0                 8   
4   0 2016-11-17          11               0                 9   

   consommation_level_4  months_number counter_type  counter_coefficient  \
0                     0              4         ELEC                    1   
1                     0              4         ELEC                    1   
2                     0              4         ELEC                    1   
3                     0              4         ELEC                    1   
4                     0             12         ELEC                    1   

   consommation_level_1  consommation_level_2  consommation_level_3  \
0                    8